In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import metrics

In [2]:
inputs_train = pd.read_csv('inputs_train.csv', index_col=0)
targets_train = pd.read_csv('targets_train.csv', index_col=0)
inputs_test = pd.read_csv('inputs_test.csv', index_col=0)
targets_test = pd.read_csv('targets_test.csv', index_col=0)

In [3]:
pd.options.display.max_columns = None

In [4]:
inputs_train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,person_home_ownership:MORTGAGE,person_home_ownership:OTHER,person_home_ownership:OWN,person_home_ownership:RENT,loan_grade:A,loan_grade:B,loan_grade:C,loan_grade:D,loan_grade:E,loan_grade:F,loan_grade:G,cb_person_default_on_file:N,cb_person_default_on_file:Y,loan_intent:DEBTCONSOLIDATION,loan_intent:EDUCATION,loan_intent:HOMEIMPROVEMENT,loan_intent:MEDICAL,loan_intent:PERSONAL,loan_intent:VENTURE,person_home_ownership:OTHER_RENT,person_age:<=22,person_age:23-28,person_age:29-33,person_age:34-37,person_age:>=38,person_income_factor,person_income:<=20K,person_income:20K-34K,person_income:34K-52.5K,person_income:52.5K-66.5K,person_income:66.5K-75.7K,person_income:>75.7K,person_emp_length:<=2,person_emp_length:3-7,person_emp_length:8-10,person_emp_length:>10,loan_amnt_factor,loan_amnt:<=3260,loan_amnt:3260-8090,loan_amnt:8090-12230,loan_amnt:12230-18440,loan_amnt:>18440,loan_int_rate_factor,loan_int_rate:<=6.488,loan_int_rate:6.488-9.692,loan_int_rate:9.692-10.76,loan_int_rate:10.76-13.964,loan_int_rate:13.964-15.388,loan_int_rate:15.388-17.88,loan_int_rate:>17.88,loan_percent_income_factor,loan_percent_income:<=6.64pct,loan_percent_income:6.64-14.9pct,loan_percent_income:14.9-16.6pct,loan_percent_income:16.6-29.9pct,loan_percent_income:29.9-31.5pct,loan_percent_income:>31.5pct
24386,31,50000,RENT,1.0,MEDICAL,B,10000,12.69,0,0.20,N,8,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,"(44715.68, 85431.36]",1,1,1,0,0,0,1,0,0,0,"(9470.0, 10160.0]",0,0,1,0,0,"(12.54, 12.896]",0,0,0,1,0,0,0,"(0.199, 0.216]",0,0,0,1,0,0
28394,27,80000,MORTGAGE,1.0,EDUCATION,B,20000,11.36,0,0.25,N,8,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,"(44715.68, 85431.36]",1,1,1,1,1,1,1,0,0,0,"(19820.0, 20510.0]",0,0,0,0,1,"(11.116, 11.472]",0,0,0,1,0,0,0,"(0.249, 0.266]",0,0,0,1,0,0
1342,25,28000,RENT,6.0,DEBTCONSOLIDATION,A,1000,7.51,0,0.04,N,3,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,"(1964.216, 44715.68]",1,1,0,0,0,0,0,1,0,0,"(465.5, 1190.0]",1,0,0,0,0,"(7.2, 7.556]",0,1,0,0,0,0,0,"(0.0332, 0.0498]",1,0,0,0,0,0
7051,21,51000,MORTGAGE,5.0,PERSONAL,A,6000,7.88,0,0.12,N,2,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,"(44715.68, 85431.36]",1,1,1,0,0,0,0,1,0,0,"(5330.0, 6020.0]",0,1,0,0,0,"(7.556, 7.912]",0,1,0,0,0,0,0,"(0.116, 0.133]",0,0,0,0,0,0
8229,26,56004,MORTGAGE,10.0,EDUCATION,C,20000,13.57,0,0.36,Y,4,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,"(44715.68, 85431.36]",1,1,1,1,0,0,0,0,1,0,"(19820.0, 20510.0]",0,0,0,0,1,"(13.252, 13.608]",0,0,0,1,0,0,0,"(0.349, 0.365]",0,0,0,0,0,1


In [5]:
inputs_train.shape

(26060, 69)

In [6]:
targets_train.shape

(26060, 1)

In [7]:
inputs_test.shape

(6516, 69)

In [8]:
targets_test.shape

(6516, 1)

In [9]:
targets_train.head()

,good_bad
24386,1
28394,1
1342,1
7051,1
8229,1


In [10]:
inputs_train_ref = inputs_train.loc[:,[
    'person_home_ownership:OWN', 
'person_home_ownership:MORTGAGE', 
'person_home_ownership:OTHER_RENT',
'cb_person_default_on_file:Y', 
'cb_person_default_on_file:N',
'loan_grade:A',  
'loan_grade:B',
'loan_grade:C', 
'loan_grade:D', 
'loan_grade:E', 
'loan_grade:F', 
'loan_grade:G',
'loan_intent:DEBTCONSOLIDATION',
'loan_intent:EDUCATION', 
'loan_intent:HOMEIMPROVEMENT',
'loan_intent:MEDICAL',
'loan_intent:PERSONAL',
'loan_intent:VENTURE',
'person_age:<=22', 
'person_age:23-28', 
'person_age:29-33', 
'person_age:34-37',  
'person_age:>=38',
'person_income:<=20K', 
'person_income:20K-34K',  
'person_income:34K-52.5K', 
'person_income:52.5K-66.5K', 
'person_income:66.5K-75.7K',  
'person_income:>75.7K',
'person_emp_length:<=2', 
'person_emp_length:3-7', 
'person_emp_length:8-10', 
'person_emp_length:>10',
'loan_amnt:<=3260', 
'loan_amnt:3260-8090', 
'loan_amnt:8090-12230', 
'loan_amnt:12230-18440',  
'loan_amnt:>18440',
'loan_int_rate:<=6.488', 
'loan_int_rate:6.488-9.692', 
'loan_int_rate:9.692-10.76', 
'loan_int_rate:10.76-13.964', 
'loan_int_rate:13.964-15.388', 
'loan_int_rate:15.388-17.88', 
'loan_int_rate:>17.88',
'loan_percent_income:<=6.64pct', 
'loan_percent_income:6.64-14.9pct', 
'loan_percent_income:14.9-16.6pct', 
'loan_percent_income:16.6-29.9pct', 
'loan_percent_income:29.9-31.5pct', 
'loan_percent_income:>31.5pct'
]]

In [11]:
ref_categories = [
    'person_home_ownership:OTHER_RENT',
'cb_person_default_on_file:Y',
'loan_grade:G',
'loan_intent:DEBTCONSOLIDATION',
'person_age:<=22',
'person_income:<=20K',
'person_emp_length:<=2',
'loan_amnt:>18440',
'loan_int_rate:>17.88',
'loan_percent_income:>31.5pct'
]

In [12]:
X_train = inputs_train_ref.drop(ref_categories, axis=1)

In [13]:
X_train.shape

(26060, 41)

In [14]:
y_train = targets_train.values.ravel()

In [15]:
y_train.shape

(26060,)

# fit the model

In [18]:
reg = LogisticRegression(max_iter=1000)

In [21]:
reg.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [22]:
reg.intercept_

array([-9.70377857])